In [1]:
import sys, os
from datetime import datetime, timedelta,date
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model

In [2]:
products_df = pd.read_csv('./../data/amazon_baby.csv')

In [3]:
products_df.shape

(183531, 3)

In [4]:
products_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [5]:
products_df.review = products_df.review.fillna('')
products_df['reviews_clean'] = products_df.review.str.replace('[^\w\s]','').fillna('')
products_df = products_df[products_df.rating != 3]

In [6]:
products_df['sentiment'] = np.where(products_df['rating'] > 3, +1, -1)

In [7]:
products_df.head()

,name,review,rating,reviews_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


In [8]:
train_indexes = pd.read_json('../data/module-2-assignment-train-idx.json')
test_indexes = pd.read_json('../data/module-2-assignment-test-idx.json')

In [9]:
train_data = products_df.iloc[train_indexes[0].tolist()]
test_data = products_df.iloc[test_indexes[0].tolist()]

In [10]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

In [11]:
train_matrix = vectorizer.fit_transform(train_data['reviews_clean'])

In [12]:
test_matrix = vectorizer.transform(test_data['reviews_clean'])

In [13]:
sentiment_model = linear_model.LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
print("Number of Coefficients of this model are = {}".format(len(sentiment_model.coef_[0])))

Number of Coefficients of this model are = 121741


In [15]:
print("Number of Coefficients of this model are = {}".format(sum(sentiment_model.coef_[0] >= 0)))

Number of Coefficients of this model are = 85924


In [16]:
sample_test_data = test_data[10:13]
sample_test_data.head()

,name,review,rating,reviews_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


In [17]:
for i in range(3):
    print("Review {} is {}".format(i, sample_test_data.iloc[i]['review']))

Review 0 is Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.
Review 1 is Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.
Review 2 is Was so excited to get this product for my baby girls bedroom!  When I got it the back is NOT STICKY at all!  Every time I walked into the bedroom I was picking up pieces off of the floor!  Very very frustrating!  Ended up having to super glue it to the wall...very disappointing.  I wouldn't waste the time or money on it.


In [18]:
sample_test_matrix = vectorizer.transform(sample_test_data['reviews_clean'])


In [19]:
sentiment_model.decision_function(sample_test_matrix)

array([  5.60827718,  -3.15921163, -10.41343225])

In [20]:
def custom_predict(model, text_sparse_matrix):
    predictions = [1 if s >=0 else -1 for s in model.decision_function(text_sparse_matrix)]
    return np.array(predictions)

In [21]:
custom_predict(sentiment_model, sample_test_matrix)

array([ 1, -1, -1])

In [22]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1])

In [23]:
def prob(x):
    return 1/(1+ np.exp(-x))

In [24]:
def predict_probs(sentiment_model, test_matrix):
    return np.array([prob(s) for s in sentiment_model.decision_function(test_matrix)])

In [25]:
predict_probs(sentiment_model, sample_test_matrix)

array([9.96346018e-01, 4.07298447e-02, 3.00255379e-05])

In [26]:
class_predictions_test_data = sentiment_model.predict(test_matrix)

In [27]:
sentiment_model.predict_proba(test_matrix)

array([[2.20460442e-01, 7.79539558e-01],
       [7.44152354e-07, 9.99999256e-01],
       [6.62670976e-02, 9.33732902e-01],
       ...,
       [5.41656421e-06, 9.99994583e-01],
       [2.47500495e-06, 9.99997525e-01],
       [1.88335064e-02, 9.81166494e-01]])

In [28]:
test_data['positive_review_prediction'] = pd.Series([sentiment_model.predict_proba(row)[0][1] for row in test_matrix])

/Users/nehiljain/.pyenv/versions/3.4.6/envs/py34/lib/python3.4/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
test_data_pos = test_data.sort_values(['positive_review_prediction'], ascending=False)

### Top 20 Positive Reviews 

In [30]:
test_data_pos[['review', 'positive_review_prediction']].head(20)

,review,positive_review_prediction
14482,This unit worked well for the first few months...,1.0
15732,"I GOT THIS FOR MY NEW GRANDDAUGHTER ,SHE LOVE ...",1.0
18112,It was the best buy for our baby. I'm so happy...,1.0
30535,"very good! Recommended moms buy, your baby is ...",1.0
26830,I have twin boys and find that very few shoppi...,1.0
31271,Love this mouse! We bought the large for my 18...,1.0
18009,"Super stretchy, easy to attach to wall, and so...",1.0
20905,This is perfect for a newborn.Pros: Soft fabri...,1.0
12823,We have had this chair for almost three years ...,1.0
32449,Works entirely like it's supposed to. The onl...,1.0


In [31]:
test_data_neg = test_data.sort_values(['positive_review_prediction'], ascending=True)

### Top 20 Negative Reviews

In [32]:
test_data_neg[['review', 'positive_review_prediction']].head(20)

,review,positive_review_prediction
9655,"I am a big fan of the ""Baby Einstein"" products...",3.454597e-11
31226,I LOVE being able to feed my baby whole food. ...,6.165371e-10
7310,"I got this pump when I was still pregnant, I d...",7.265702e-10
17222,We choose the Fisher Price swing because they ...,3.101836e-09
17985,"muito boa para guardar os bichos de pelucia, p...",7.318266e-09
13752,Exactly what I needed and ordered and came sup...,2.634153e-08
13572,"Smells wonderful, nice and creamy so won't dry...",4.120121e-08
3747,I love the Diaper Genie! I received this as a...,1.023396e-07
394,Works great - no smells. LOVE that it uses re...,1.140153e-07
30538,grandson really loves the stuff you put in thi...,1.223347e-07


### Evaluation

In [33]:
def accuracy(truth_labels, predicted_labels):
    return float(sum(truth_labels == predicted_labels)/len(truth_labels))

In [34]:
test_data['predicted_class'] = pd.Series(sentiment_model.predict(test_matrix))

/Users/nehiljain/.pyenv/versions/3.4.6/envs/py34/lib/python3.4/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
